### Transformar JSON y creación de archivos de tiempos de espera

In [2]:
# Instancia

file = '20-100-10-0'

In [3]:
# Imports

import os
import csv
import json
import math
import pandas as pd

# Carpeta de tiempos de espera

nombre_directorio = 'waitingTimes/' + file
os.mkdir(nombre_directorio)

# 1. Script para crear array de estados de cada vehículo

In [4]:
def readJson(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def writeJson(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=2)

def calculateCost(source, target, distances):
    for distance in distances:
        if distance['source'] == source and distance['target'] == target:
            return max(distance['value'], 0)
    return None

# Implementación del algoritmo Bellman Ford
def bellman_ford(graph, source):
    distance = {vertex: math.inf for vertex in graph}
    distance[source] = 0

    for _ in range(len(graph) - 1):
        for vertex in graph:
            for neighbor, weight in graph[vertex]:
                if distance[vertex] + weight < distance[neighbor]:
                    distance[neighbor] = distance[vertex] + weight

    return distance

def transformSolutions(instances, solutions):
    distances = instances['distances']
    vehicles, graph = {}, {}

    for distance in distances:
        source, target, value = distance['source'], distance['target'], distance['value']
        graph.setdefault(source, []).append((target, value))
        
    solutions['routes'] = sorted(solutions['routes'], key=lambda x: x['vehicle']['id'])
    
    for route in solutions['routes']:
        vehicle, jobs = route['vehicle'], route['route']
        initialPos = vehicle['initialPos']
        route_length = len(jobs)
        for job in jobs:
            if 'job' in job:
                endPosJob = job['job']['endPos']
                vehicles.setdefault(endPosJob, []).append(vehicle)
                
        route['vehicleInfo'] = {'route_length': route_length}
                

    for pos in vehicles:
        if pos in graph:
            vehicles[pos].sort(key=lambda x: x['priority'])  # Ordena los vehículos por prioridad
            graph[pos].append((pos, -vehicles[pos][0]['priority']))  # Agrega movimiento con prioridad negativa

    mixed_trayectories = []
    id_counter = 0

    for route in solutions['routes']:
        vehicle, jobs = route['vehicle'], route['route']
        initialPos, vehicleInitTime = vehicle['initialPos'], vehicle['time']
        desplazamientos, accumulatedCost = [], vehicleInitTime
        moveCost_empty = 0
        moveCost_working = 0
        initJobCost = 0
        endJobCost = 0
        currentPos = initialPos
        desplazamientos.append({'id': id_counter, 
                                'status': 'initVehicle', 
                                'currentPos': currentPos, 
                                'initialPos': initialPos,
                                'endPos': initialPos,
                                'vehicleInitTime': vehicleInitTime,
                                'vehicle': vehicle['id'],
                                'accumulatedCost': 0})
        id_counter += 1

        for jobIndex, job in enumerate(jobs):
            initialJobTime, endJobTime = job['job']['initialTime'], job['job']['endTime']
            initialPosJob, endPosJob = job['job']['initialPos'], job['job']['endPos']
            moveCost = calculateCost(initialPos, initialPosJob, distances)

            if moveCost is not None:
                moveCost_empty += moveCost
                desplazamientos.append({'id': id_counter, 
                                        'status': 'movingEmptyVehicle', 
                                        'moveCost': moveCost,
                                        'vehicle': vehicle['id'],
                                        'currentPos': currentPos,
                                        'initialPos': initialPos,
                                        'endPos': initialPosJob,
                                        'accumulatedCost': accumulatedCost})
                
                accumulatedCost += moveCost
                initialPos = initialPosJob
                currentPos = initialPosJob

            jobMoveCost = calculateCost(initialPosJob, endPosJob, distances)

            if jobMoveCost is not None:
                initJobCost += initialJobTime
                endJobCost += endJobTime
                moveCost_working += jobMoveCost
                
                desplazamientos.append({'id': id_counter + 1,
                                        'status': 'initJob',
                                        'currentPos': currentPos,
                                        'endPos': currentPos,
                                        'initJobCost': initialJobTime,
                                        'endJobCost': endJobTime,
                                        'totalJobCost': initialJobTime + jobMoveCost + endJobTime,
                                        'vehicle': vehicle['id'],
                                        'accumulatedCost': accumulatedCost})
                
                accumulatedCost += initialJobTime
                
                desplazamientos.append({'id': id_counter + 2,
                                        'status': 'movingWorkingVehicle',
                                        'currentPos': currentPos,
                                        'initJobCost': initialJobTime,
                                        'endJobCost': endJobTime, 
                                        'initialPos': initialPosJob, 
                                        'endPos': endPosJob,
                                        'moveCost': jobMoveCost, 
                                        'totalJobCost': initialJobTime + jobMoveCost + endJobTime,
                                        'vehicle': vehicle['id'],
                                        'accumulatedCost': accumulatedCost})
                
                accumulatedCost += jobMoveCost
                currentPos = endPosJob
                
                desplazamientos.append({'id': id_counter + 3,
                                        'status': 'endingJob',
                                        'currentPos': currentPos,
                                        'endPos': currentPos,
                                        'initJobCost': initialJobTime,
                                        'endJobCost': endJobTime, 
                                        'totalJobCost': initialJobTime + jobMoveCost + endJobTime,
                                        'vehicle': vehicle['id'], 
                                        'accumulatedCost': accumulatedCost})
                
                accumulatedCost += endJobTime
                initialPos = endPosJob
            id_counter += 4

        # Agregar el elemento 'finished' al final de la trayectoria
        desplazamientos.append({'id': id_counter, 'status': 'finished','vehicle': vehicle['id'], 'accumulatedCost': accumulatedCost})
        id_counter += 1
        
        route['trayectoria'] = {'desplazamientos': desplazamientos, 'totalCost': accumulatedCost}
        mixed_trayectories.extend(desplazamientos)
               
        
        # Agregar información de costos al array vehiclesInfo
        route['vehicleInfo'].update ({
            'vehicleId': vehicle['id']+1,
            'moveCost_empty': moveCost_empty,
            'moveCost_working': moveCost_working,
            'initJobCost': initJobCost,
            'endJobCost': endJobCost,
            'totalCost_expected': moveCost_empty + moveCost_working + initJobCost + endJobCost + vehicleInitTime,
            'relMoveEmpty': round((moveCost_empty/(moveCost_empty + moveCost_working + initJobCost + endJobCost)), 4),
            'relMoveWorking': round((moveCost_working/(moveCost_empty + moveCost_working + initJobCost + endJobCost)), 4),
            'relStoppedTime': round(((initJobCost + endJobCost)/(moveCost_empty + moveCost_working + initJobCost + endJobCost)), 4)
        })

    solutions['mixedTrayectories'] = mixed_trayectories
    

    for route in solutions['routes']:
        initial_id = route['trayectoria']['desplazamientos'][0]['id']
        for desplazamiento in route['trayectoria']['desplazamientos']:
            desplazamiento['id'] -= initial_id

    return solutions


# Función principal
def main():
    instances = readJson('instances/instance-' + file + '.json')
    solutions = readJson('solutions/solution-' + file + '.json')
    solutions = transformSolutions(instances, solutions)
    writeJson(solutions, 'solutionsComplete/solutionComplete-' + file + '.json')

# Ejecuta la función principal si el script se ejecuta directamente
if __name__ == "__main__":
    main()


# 2. Script para obtener las posiciones de los vehículos

In [5]:
def write_vehicle_positions_to_csv(solutions):
    with open('waitingTimes/' + file +'/1vehicle_positions-' + file + '.csv', 'w', newline='') as csvfile:
        fieldnames = ['id','vehicle_id', 'accumulatedCost', 'position']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        id = 0
        for route in solutions['routes']:
            vehicle_id = route['vehicle']['id']
            for desplazamiento in route['trayectoria']['desplazamientos']:
                accumulatedCost = desplazamiento['accumulatedCost']
                position = desplazamiento.get('currentPos', None)  # Usamos get para obtener el valor de 'currentPos' o None si no está presente
                if position is not None:
                    writer.writerow({'id': id, 'vehicle_id': vehicle_id, 'accumulatedCost': accumulatedCost, 'position': position})
                    id += 1
                    
# Después de llamar a transformSolutions, llama a esta nueva función
def main():
    instances = readJson('instances/instance-' + file + '.json')
    solutions = readJson('solutions/solution-' + file + '.json')
    solutions = transformSolutions(instances, solutions)
    writeJson(solutions, 'solutionsComplete/solutionComplete-' + file + '.json')

    # Llama a la nueva función para escribir las posiciones de los vehículos en un CSV
    write_vehicle_positions_to_csv(solutions)

# Ejecuta la función principal si el script se ejecuta directamente
if __name__ == "__main__":
    main()


# 3. Script para obtener las posiciones de los vehículos en cada instante de tiempo

In [6]:
def expand_positions(input_file, output_file):
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        rows = list(reader)

    expanded_rows = []
    current_id = 0
    for i in range(len(rows) - 1):
        current_row = rows[i]
        next_row = rows[i + 1]
        vehicle_id = int(current_row['vehicle_id'])
        accumulated_cost = int(current_row['accumulatedCost'])
        current_position = int(current_row['position'])
        next_position = int(next_row['position'])
        for j in range(accumulated_cost, int(next_row['accumulatedCost'])):
            expanded_rows.append({
                'id': current_id,
                'vehicle_id': vehicle_id,
                'accumulatedCost': j,
                'position': current_position if j < int(next_row['accumulatedCost']) else next_position
            })
            current_id += 1

    expanded_rows.append({
        'id': current_id,
        'vehicle_id': int(rows[-1]['vehicle_id']),
        'accumulatedCost': int(rows[-1]['accumulatedCost']),
        'position': int(rows[-1]['position'])
    })

    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['id', 'vehicle_id', 'accumulatedCost', 'position']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(expanded_rows)

# Ejemplo de uso
expand_positions('waitingTimes/' + file +'/1vehicle_positions-' + file + '.csv', 
                 'waitingTimes/' + file +'/2complete_vehicle_positions-' + file + '.csv')


# 4. Script para los tiempos de acciones de vehículos

In [7]:
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['time1', 'time2', 'vehicleid', 'currentPos', 'endPos', 'status']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        next_accumulated_costs = [item['accumulatedCost'] for item in data[1:]]  # Get accumulatedCost of next elements
        for item, next_accumulated_cost in zip(data, next_accumulated_costs):
            
                writer.writerow({'time1': item['accumulatedCost'],
                                 'time2': next_accumulated_cost,
                                 'vehicleid': item['vehicle'],
                                 'currentPos': item.get('currentPos', ''),
                                 'endPos': item.get('endPos', ''),
                                 'status': item.get('status', '')})

def find_matching_positions(data):
    positions = {}
    for item in data:
        if item['status'] not in ['initVehicle', 'initJob', 'endingJob', 'finished']:
            vehicle_id = item['vehicle']
            current_pos = item.get('currentPos', '')
            end_pos = item.get('endPos', '')
            if current_pos and end_pos:
                if current_pos not in positions:
                    positions[current_pos] = set()
                positions[current_pos].add(vehicle_id)
    matching_pairs = []
    for pos, vehicles in positions.items():
        if len(vehicles) > 1:
            matching_pairs.append((pos, vehicles))
    return matching_pairs

# Lee el archivo JSON
data = read_json('solutionsComplete/solutionComplete-' + file + '.json')
    
# Obtiene el array mixedTrayectories
mixed_trayectories = data.get('mixedTrayectories', [])
    
# Escribe los datos en un archivo CSV
write_csv(mixed_trayectories, 'waitingTimes/' + file +'/3mixed_trayectories_filtered-' + file + '.csv')
    
# Encuentra las parejas de filas con vehiclesid diferentes que tengan un currentPos igual a un endPos
matching_pairs = find_matching_positions(mixed_trayectories)
print("Parejas de filas con vehiclesid diferentes que tienen un currentPos igual a un endPos:")
for pos, vehicles in matching_pairs:
    if len(vehicles) > 1:
        print(f"Posición: {pos}, Vehículos: {vehicles}")


Parejas de filas con vehiclesid diferentes que tienen un currentPos igual a un endPos:
Posición: 96, Vehículos: {0, 2, 7}
Posición: 75, Vehículos: {1, 3}
Posición: 51, Vehículos: {2, 7}
Posición: 36, Vehículos: {8, 2}
Posición: 48, Vehículos: {4, 6}


# 5. Script para obtener posiciones en conflicto

In [8]:
# Leer el archivo CSV
df = pd.read_csv('waitingTimes/' + file +'/3mixed_trayectories_filtered-' + file + '.csv')

# Filas que tienen currentPos y endPos
df_conflict = df.dropna(subset=['currentPos', 'endPos']).copy()

# Quitar los decimales convirtiendo los valores a enteros
df_conflict['currentPos'] = df_conflict['currentPos'].astype(int).astype(str)
df_conflict['endPos'] = df_conflict['endPos'].astype(int).astype(str)

# Crear DataFrames de conflictos sin considerar el vehicleid
endPos_conflicts = df_conflict[df_conflict.duplicated('endPos', keep=False)]
currentPos_conflicts = df_conflict[df_conflict.duplicated('currentPos', keep=False)]
cross_conflicts_1 = df_conflict[df_conflict['currentPos'].isin(df_conflict['endPos'])]
cross_conflicts_2 = df_conflict[df_conflict['endPos'].isin(df_conflict['currentPos'])]

# Función para agregar conflictos si el vehicleid es diferente
def add_conflicts(conflict_df, conflict_set):
    for idx, row in conflict_df.iterrows():
        conflicting_rows = conflict_df[(conflict_df['endPos'] == row['endPos']) | 
                                       (conflict_df['currentPos'] == row['currentPos']) | 
                                       (conflict_df['currentPos'] == row['endPos']) | 
                                       (conflict_df['endPos'] == row['currentPos'])]
        for _, conflict_row in conflicting_rows.iterrows():
            if row['vehicleid'] != conflict_row['vehicleid']:
                conflict_set.add((row['currentPos'], row['endPos']))
                conflict_set.add((conflict_row['currentPos'], conflict_row['endPos']))

# Crear un conjunto para detección de conflictos
conflict_set = set()

# Añadir conflictos considerando vehicleid
add_conflicts(endPos_conflicts, conflict_set)
add_conflicts(currentPos_conflicts, conflict_set)
add_conflicts(cross_conflicts_1, conflict_set)
add_conflicts(cross_conflicts_2, conflict_set)

# Filtrar las filas en conflicto
conflict_rows = df_conflict[
    df_conflict.apply(lambda x: (x['currentPos'], x['endPos']) in conflict_set, axis=1)
]

# Guardar las filas en conflicto en un nuevo CSV
conflict_rows.to_csv('waitingTimes/' + file +'/4conflict_rows1-' + file + '.csv', index=False)


# 6. Script para obtener posiciones en conflicto de diferentes vehículos

In [9]:
# Leer el archivo CSV
df = pd.read_csv('waitingTimes/' + file +'/4conflict_rows1-' + file + '.csv')

# Filas que tienen currentPos y endPos
df_conflict = df.dropna(subset=['currentPos', 'endPos', 'time1', 'time2']).copy()

# Quitar los decimales convirtiendo los valores a enteros
df_conflict['currentPos'] = df_conflict['currentPos'].astype(int).astype(str)
df_conflict['endPos'] = df_conflict['endPos'].astype(int).astype(str)

# Convertir los tiempos a enteros
df_conflict['time1'] = df_conflict['time1'].astype(int)
df_conflict['time2'] = df_conflict['time2'].astype(int)

# Función para agregar conflictos si hay solapamiento de tiempo y el vehicleid es diferente
def add_conflicts(conflict_df, conflict_pairs):
    for idx, row in conflict_df.iterrows():
        conflicting_rows = conflict_df[
            (conflict_df['currentPos'] == row['currentPos']) |
            (conflict_df['endPos'] == row['endPos']) |
            (conflict_df['currentPos'] == row['endPos']) |
            (conflict_df['endPos'] == row['currentPos'])
        ]
        for _, conflict_row in conflicting_rows.iterrows():
            if row['vehicleid'] != conflict_row['vehicleid']:
                # Check for time overlap
                if (row['time1'] <= conflict_row['time2'] and row['time2'] >= conflict_row['time1']):
                    conflict_pairs.append((row, conflict_row))

# Crear una lista para las parejas de conflictos
conflict_pairs = []

# Añadir conflictos considerando vehicleid y solapamiento de tiempo
add_conflicts(df_conflict, conflict_pairs)

# Crear un DataFrame para las parejas de conflictos
conflict_rows = pd.DataFrame()
for row1, row2 in conflict_pairs:
    conflict_rows = pd.concat([conflict_rows, pd.DataFrame([row1]), pd.DataFrame([row2])])

# Eliminar duplicados en caso de que una fila se repita en varias parejas
conflict_rows = conflict_rows.drop_duplicates()

# Guardar las filas en conflicto en un nuevo CSV
conflict_rows.to_csv('waitingTimes/' + file +'/5conflict_rows2-' + file + '.csv', index=False)


# 7. Script para obtener grupos de conflictos entre vehículos distintos

In [11]:
# Leer el archivo CSV original
df = pd.read_csv('waitingTimes/' + file +'/5conflict_rows2-' + file + '.csv')

# Filtrar las filas que tienen currentPos, endPos, time1 y time2 no nulos
df = df.dropna(subset=['currentPos', 'endPos', 'time1', 'time2']).copy()

# Quitar los decimales convirtiendo los valores a enteros
df['currentPos'] = df['currentPos'].astype(int)
df['endPos'] = df['endPos'].astype(int)
df['time1'] = df['time1'].astype(int)
df['time2'] = df['time2'].astype(int)

# Inicializar la columna de grupo de conflictos
df['conflict_group'] = -1

# Función para encontrar conflictos y asignarles un ID de grupo
def find_and_assign_conflicts(df):
    group_id = 0
    for idx, row in df.iterrows():
        if df.at[idx, 'conflict_group'] == -1:  # Si aún no ha sido asignado a un grupo
            df.at[idx, 'conflict_group'] = group_id
            # Usar DFS para asignar el mismo group_id a todos los conflictos relacionados
            stack = [idx]
            while stack:
                current_idx = stack.pop()
                current_row = df.loc[current_idx]
                # Encontrar filas en conflicto
                conflicts = df[(df['vehicleid'] != current_row['vehicleid']) & (
                    ((df['currentPos'] == current_row['currentPos']) |
                     (df['endPos'] == current_row['endPos']) |
                     (df['currentPos'] == current_row['endPos']) |
                     (df['endPos'] == current_row['currentPos'])) & 
                    (df['time1'] <= current_row['time2']) & 
                    (df['time2'] >= current_row['time1']))]
                for conflict_idx in conflicts.index:
                    if df.at[conflict_idx, 'conflict_group'] == -1:
                        df.at[conflict_idx, 'conflict_group'] = group_id
                        stack.append(conflict_idx)
            group_id += 1

# Encontrar y asignar los grupos de conflictos
find_and_assign_conflicts(df)

# Leer el archivo CSV que contiene las prioridades de los vehículos
priorities_df = pd.read_csv('waitingTimes/' + file +'/6vehicle_priorities-' + file + '.csv')

# Fusionar los DataFrames por la columna 'vehicleid'
df = pd.merge(df, priorities_df, how='left', left_on='vehicleid', right_on='Vehicle ID')

# Eliminar la columna 'Vehicle ID' después de la fusión
df.drop(columns=['Vehicle ID'], inplace=True)

# Ordenar las columnas según lo solicitado
df = df[['time1', 'time2', 'vehicleid', 'Priority', 'currentPos', 'endPos', 'status', 'conflict_group']]

# Guardar el DataFrame con los grupos de conflictos y la columna de prioridad en un nuevo CSV
df.to_csv('waitingTimes/' + file +'/7conflict_rows3-' + file + '.csv', index=False)
